# Data Exploration and Visualisation

### Part One : Using How2 Dataset for YouTube videos IDs, transcriptions and description summaries. 

* Construct databases of video transcriptions and human made descriptions
* Also, download audios of 400 videos

In [161]:
import pandas as pd

In [2]:
# Load the test dataset into a pandas data frame
# Load the descriptions into a pandas data frame
# Load the transcriptions into into a pandas data frame 
desctable = pd.read_csv('speech_data/text/sum_cv/desc.tok.txt', header=None, skipinitialspace=True, names=["a"])
desctable.head()
filename = 'speech_data/text/sum_cv/tran.tok.txt'
with open(filename, 'r') as f:
    data = f.read().replace('\n','%%%')
trantable = pd.read_csv(pd.compat.StringIO(data), sep="%%%", header=None)
trantable = trantable.T
trantable.drop(trantable.tail(1).index,inplace=True)
trantable.columns=['a']

/Users/dayangraham/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,a
0,-xd1aAlPXqs after you 've done at least six to...
1,"KtMjOT6fDrw on behalf of expert village , my n..."
2,ehbFyYlcEhc hi ! this is david jackel on behal...
3,lcw8f2od6z8 the other way we can do cucumbers ...
4,G-VRHmkiqtc my name is anthony maddaloni and i...


In [3]:
# Split id and description
desctable['id'] = desctable.apply(lambda row: str(row.a).split(" ")[0], axis = 1) 
desctable['desc'] = desctable.apply(lambda row: ' '.join(str(row.a).split(" ")[1:]), axis = 1) 
desctable.drop('a',1,inplace=True)

,id,desc
0,-xd1aAlPXqs,learn the sivananda yoga single right leg rais...
1,KtMjOT6fDrw,learn how to apply hanger hooks for your woodc...
2,ehbFyYlcEhc,learn about how hand washing can help prevent ...
3,lcw8f2od6z8,how to julienne cucumbers to make kimchi for k...
4,G-VRHmkiqtc,in order to put photographic emulsion on water...


In [4]:
# Split id and transcription
trantable['id'] = trantable.apply(lambda row: row.a.split(" ")[0], axis = 1) 
trantable['tran'] = trantable.apply(lambda row: ' '.join(row.a.split(" ")[1:]), axis = 1) 
trantable.drop('a',1,inplace=True)

,id,tran
0,-xd1aAlPXqs,after you 've done at least six to twelve roun...
1,KtMjOT6fDrw,"on behalf of expert village , my name is husai..."
2,ehbFyYlcEhc,hi ! this is david jackel on behalf of expert ...
3,lcw8f2od6z8,the other way we can do cucumbers which is als...
4,G-VRHmkiqtc,my name is anthony maddaloni and i 'm going to...


In [5]:
# Join tables on ID to create a single table 
conctable = pd.merge(desctable,trantable,on="id")
conctable.head()

In [115]:
# Install dependencies to get audio from YouTube
!pip -q install wget youtube-dl wget 

In [7]:
# Take the first 400 YouTube video IDs
youtube_ids = conctable['id'].tolist()
shortlist = youtube_ids[:400]

In [9]:
# Loop over the 400 YouTube videos
# Save each video's audio as 8000Hz wav
for YOUTUBE_ID in shortlist:
    !youtube-dl --extract-audio --audio-format wav --quiet --output "{YOUTUBE_ID}_FULL.%(ext)s" https://www.youtube.com/watch\?v\={YOUTUBE_ID}
    !ffmpeg -loglevel panic -y -i {YOUTUBE_ID}_FULL.wav -acodec pcm_s16le -ac 1 -ar 8000 {YOUTUBE_ID}.wav
    !rm {YOUTUBE_ID}_FULL.wav

rm: illegal option -- k
usage: rm [-f | -i] [-dPRrvW] file ...
       unlink file
rm: illegal option -- T
usage: rm [-f | -i] [-dPRrvW] file ...
       unlink file
rm: illegal option -- 0
usage: rm [-f | -i] [-dPRrvW] file ...
       unlink file
rm: illegal option -- N
usage: rm [-f | -i] [-dPRrvW] file ...
       unlink file


## Data exploration and statistics

In [481]:
exploretable = conctable.head(400)

In [482]:
exploretable['intersection'] = exploretable.apply(lambda row: [value for value in row.desc.split(" ") if value in row.tran.split(" ")] , axis=1)
exploretable['descnottran'] = exploretable.apply(lambda row: [value for value in row.desc.split(" ") if value not in row.tran.split(" ")] , axis=1)

/Users/dayangraham/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/dayangraham/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [490]:
exploretable['rouge1-f'] = exploretable.apply(lambda row:  rouge.get_scores(row.desc, row.tran)[0]['rouge-1']['f'], axis=1)
exploretable['rouge1-r'] = exploretable.apply(lambda row:  rouge.get_scores(row.desc, row.tran)[0]['rouge-1']['r'], axis=1)
exploretable['rouge1-p'] = exploretable.apply(lambda row:  rouge.get_scores(row.desc, row.tran)[0]['rouge-1']['p'], axis=1)
exploretable['rouge2-f'] = exploretable.apply(lambda row:  rouge.get_scores(row.desc, row.tran)[0]['rouge-2']['f'], axis=1)
exploretable['rouge2-r'] = exploretable.apply(lambda row:  rouge.get_scores(row.desc, row.tran)[0]['rouge-2']['r'], axis=1)
exploretable['rouge2-p'] = exploretable.apply(lambda row:  rouge.get_scores(row.desc, row.tran)[0]['rouge-2']['p'], axis=1)
exploretable['rougel-f'] = exploretable.apply(lambda row:  rouge.get_scores(row.desc, row.tran)[0]['rouge-l']['f'], axis=1)
exploretable['rougel-r'] = exploretable.apply(lambda row:  rouge.get_scores(row.desc, row.tran)[0]['rouge-l']['r'], axis=1)
exploretable['rougel-p'] = exploretable.apply(lambda row:  rouge.get_scores(row.desc, row.tran)[0]['rouge-l']['p'], axis=1)

/Users/dayangraham/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/dayangraham/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/dayangraham/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [492]:
exploretable.head(75)

,id,desc,tran,intersection,descnottran,rouge1-f,rouge1-r,rouge1-p,rouge2-f,rouge2-r,rouge2-p,rougel-f,rougel-r,rougel-p
0,-xd1aAlPXqs,learn the sivananda yoga single right leg rais...,after you 've done at least six to twelve roun...,"[the, sivananda, yoga, single, right, leg, in,...","[learn, raise, free, exercise, video]",0.069869,0.037037,0.615385,0.026432,0.013953,0.250000,0.132231,0.074074,0.615385
1,KtMjOT6fDrw,learn how to apply hanger hooks for your woodc...,"on behalf of expert village , my name is husai...","[how, to, hooks, your, this, expert, wood, .]","[learn, apply, hanger, for, woodcarving, proje...",0.088050,0.050725,0.333333,0.000000,0.000000,0.000000,0.123711,0.078947,0.285714
2,ehbFyYlcEhc,learn about how hand washing can help prevent ...,hi ! this is david jackel on behalf of expert ...,"[about, hand, washing, can, prevent, you, from...","[learn, how, help, free, health, video, series]",0.065753,0.034682,0.631579,0.016529,0.008696,0.166667,0.114943,0.064516,0.526316
3,lcw8f2od6z8,how to julienne cucumbers to make kimchi for k...,the other way we can do cucumbers which is als...,"[how, to, cucumbers, to, make, and, about, thi...","[julienne, kimchi, for, korean, kimchi, rice, ...",0.083770,0.048193,0.320000,0.010582,0.006061,0.041667,0.099174,0.060606,0.272727
4,G-VRHmkiqtc,in order to put photographic emulsion on water...,my name is anthony maddaloni and i 'm going to...,"[in, to, put, photographic, emulsion, on, wate...","[order, ]",0.106667,0.056738,0.888889,0.054054,0.028571,0.500000,0.188235,0.105263,0.888889
5,_Q4AVRIkwAQ,combining bleeding and cupping methods in acup...,"in this episode , we 're actually going to use...","[bleeding, and, cupping, in, acupuncture, can,...","[combining, methods, help, certain, types, tre...",0.074866,0.040580,0.482759,0.005376,0.002907,0.035714,0.106509,0.062500,0.360000
6,lohZUjGk9bs,what terms are necessary for an umpire to know...,"alright , some of the terminology is balls and...","[an, to, ?, the, terminology, to, a, in, this,...","[what, terms, are, necessary, for, umpire, kno...",0.093264,0.053892,0.346154,0.010471,0.006024,0.040000,0.139130,0.087912,0.333333
7,fC82uLet6b4,what video cards are used on laptops ? learn a...,hi i 'm james adams i 'm here on behalf of exp...,"[what, video, cards, are, on, about, different...","[used, laptops, ?, learn, types, free, clip]",0.058212,0.030435,0.666667,0.025052,0.013072,0.300000,0.111111,0.061111,0.611111
8,N1JcMtqrqTA,basics on how to repair and tune a piano ; lea...,on behalf of expertvillage.com this is tom flo...,"[on, to, and, a, piano, this, and, in, this, f...","[basics, how, repair, tune, ;, learn, more, vi...",0.124514,0.070485,0.533333,0.007843,0.004425,0.034483,0.135135,0.081301,0.400000
9,y-MG69Ejzqc,the hero yoga pose has a very strong stance . ...,"so to position the feet as our foundation , al...","[the, hero, yoga, a, strong, ., on, this, is, ...","[pose, has, very, stance, learn, tips, how, do...",0.119816,0.067708,0.520000,0.027907,0.015707,0.125000,0.152542,0.093750,0.409091
